In [246]:
import sys
import numpy as np
import pandas as pd
import PIL
import heapq
from typing import Tuple, Iterable, Set
print(sys.version)
print(np.__version__)
print(pd.__version__)
print(PIL.__version__)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
1.25.2
2.1.0
10.0.0


In [247]:
from Map import Map_Obj

In [248]:
task1map = Map_Obj(task=1)
task2map = Map_Obj(task=2)
task3map = Map_Obj(task=3)
task4map = Map_Obj(task=4)
task5map = Map_Obj(task=5)

In [249]:
#SquareGrid class
#takes in a map object and creates a grid of the map
#map.int_map is a numpy array of the map with -1 as walls and 1 to 4 is the different cost of terrain
class Graph:
    """
    A simple square grid map class for pathfinding algorithms. 

    Methods
    -------
    in_bounds(id: list[int, int]) -> bool  
        Returns True if the given id is within the bounds of the map.
    passable(id: list[int, int]) -> bool
        Returns True if the given id is passable.
    neighbors(id: list[int, int]) -> Iterable[Tuple[int, int]]
        Returns the neighbors of the given id.
    cost(from_node: list[int, int], to_node: list[int, int]) -> int
        Returns the cost of moving from the given id to the given id.
    """
    def __init__(self, map):
        self.map = map.int_map
        self.start=map.get_start_pos()
        self.goal=map.get_goal_pos()
        self.width = map.int_map.shape[1]
        self.height = map.int_map.shape[0]
        self.weights: dict[Tuple[int, int], int] = {}

    def in_bounds(self, id: Tuple[int, int]) -> bool:
        (x, y) = id
        return 0 <= x < self.height and 0 <= y < self.width

    def passable(self, id: Tuple[int, int]) -> bool:
        return self.map[id[0], id[1]] != -1

    def neighbors(self, id: Tuple[int, int]) -> Iterable[Tuple[int, int]]:
        (x, y) = id
        neighbors = [(x + 1, y), (x - 1, y), (x, y - 1), (x, y + 1)]
        if (x + y) % 2 == 0:
            neighbors.reverse()
        neighbors = filter(self.in_bounds, neighbors)
        neighbors = filter(self.passable, neighbors)
        return neighbors
    
    def cost(self, to_node: Tuple[int, int]) -> int:
        return self.map[to_node[0], to_node[1]]

In [250]:
#priority queue
class PriorityQueue:
    """
    A simple priority queue class for pathfinding algorithms.

    Methods
    -------
    empty() -> bool  
        Returns True if the queue is empty.
    put(item: list[int,int], priority: int)
        Puts the given item into the queue with the given priority.
    get() -> list[int,int]
        Returns the item with the lowest priority.
    """
    def __init__(self):
        self.elements: list[tuple[int, Tuple[int,int]]] =[]
        
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: Tuple[int,int], priority: int):
        heapq.heappush(self.elements, (priority, item))
        
    def get(self) -> Tuple[int,int]:
        return heapq.heappop(self.elements)[1]

In [251]:
def heuristic(a: list, b: list)-> float:
    """
    Heuristic function h

    parameters
    ---------
    a: list 
        current position
    b: list
        goal position
    returns
    -------
    float
        heuristic value
        manhattan distance from current position to goal position
    """
    x1, y1 = a[0], a[1]
    x2, y2 = b[0], b[1]
    distance=abs(x1 - x2) + abs(y1 - y2)
    return distance

In [252]:
#implementing A* algorithm 
def a_star_search(graph: Graph) -> Tuple[list[Tuple[int, int]], int]:
    """
    A* search algorithm

    parameters
    ---------
    graph: SquareGrid
        map object

    returns
    -------
    Tuple[list[list[int, int]], int]
        path: list of positions
        cost: cost of the path
    """
    start = tuple(graph.start)
    goal = tuple(graph.goal)
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from: dict[Tuple[int, int], Tuple[int, int]] = {}
    cost_so_far: dict[Tuple[int, int], int] = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()

        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(goal, next)
                frontier.put(next, priority)
                came_from[next] = current
    
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path, cost_so_far[goal]



In [253]:
def assignment1(map: Map_Obj):
    """
    Assignment 1
    prints and shows the path and cost of the path

    parameters
    ---------
    map: Map_Obj
        map object
    """
    Graph = SquareGrid(map)
    print("start",Graph.start,"goal", Graph.goal)
    path, cost = a_star_search(Graph)
    print("Steps",len(path))
    print("cost",cost)
    print(path)
    map_int, map_str = map.get_maps()
    for i in range(len(path)):
        map_str[path[i][0]][path[i][1]]='@'
    for i in range(len(map_str)):
        print(map_str[i])
    map.show_map()


In [254]:
assignment1(task1map)
assignment1(task2map)
assignment1(task3map)
assignment1(task4map)

start [27, 18] goal [40, 32]
Steps 38
cost 37
[(27, 18), (27, 19), (27, 20), (27, 21), (27, 22), (27, 23), (27, 24), (27, 25), (27, 26), (27, 27), (28, 27), (28, 28), (28, 29), (28, 30), (28, 31), (28, 32), (28, 33), (28, 34), (28, 35), (28, 36), (28, 37), (29, 37), (30, 37), (31, 37), (32, 37), (33, 37), (34, 37), (35, 37), (36, 37), (37, 37), (38, 37), (39, 37), (40, 37), (40, 36), (40, 35), (40, 34), (40, 33), (40, 32)]
[' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ']
[' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' . ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ']
[' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # ' ' # 